In [0]:
import gensim.downloader as api
import numpy as np
from time import time
import re
import nltk
nltk.download('stopwords')
import itertools
import string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.models import Model
from keras.layers import LSTM, Input, Dot, Softmax, Multiply, Concatenate, Subtract, Dense, Lambda, Embedding, Dropout
from keras.layers.wrappers import Bidirectional
from keras import backend as K
from keras.optimizers import Adadelta
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import Dense
from keras.optimizers import SGD, Adam
import keras

#download the pre-trained model from gensim
word_embed_model = api.load('glove-wiki-gigaword-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


[==================================================] 100.0% 376.1/376.1MB downloaded


##The code is run on the Google colab, so the notebook is linked with my Google drive to read training, testing, development set

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


read all files from Google drive

In [0]:
import pandas as pd

train_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train4.csv')
dev_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/devset.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/testset3.csv')
'''
train_df = pd.read_csv('/content/gdrive/My Drive/esim_output/num_train.csv')
dev_df = pd.read_csv('/content/gdrive/My Drive/esim_output/num_dev.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/esim_output/num_test.csv')
embeddings = np.load('/content/gdrive/My Drive/esim_output/embeddings.npy')'''
print(train_df.shape)
print(dev_df.shape)
print(test_df.shape)

(150000, 3)
(120581, 6)
(584843, 6)


In [0]:
# the fucntion is used to changed the string labels into numbers
def change_label(df):
  labels = df['label']
  new_labels = []
  for row in labels:
    if row == 'SUPPORTS':
      new_labels.append(0)
    elif row == 'REFUTES':
      new_labels.append(1)
    else:
      new_labels.append(2)
  df['label'] = pd.Series(new_labels)

change_label(train_df)
change_label(dev_df)

In [0]:
# the function is to do the same process as the data retriever for the claims
def process_text(raw_text):
    text = str(raw_text)
    text = re.sub('_', ' ', text)
    text = re.sub('-LRB-', ' ', text)
    text = re.sub('-RRB-', ' ', text)
    text = re.sub('-LSB-', ' ', text)
    text = re.sub('-RSB-', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.split()
    return text

The vocab stores all the vocabulary in all sets and the index stores the position of the word in the embedding matrix.

In [0]:
vocab = {}
vocab_index = ['<unk>']

#substitute the sentence in the train data into the index representation
for index, row in train_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    train_df.at[index, sentence] = number_rep
    
#substitute the sentence in the test data into the index representation
for index, row in test_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    test_df.at[index, sentence] = number_rep
    
#substitute the sentence in the dev data into the index representation   
for index, row in dev_df.iterrows():
  if index%100000 == 0:
    print(index)
  for sentence in ['claim', 'evidence']:
    number_rep = []
    for word in process_text(row[sentence]):
      if word in stopwords.words('english') and word not in word_embed_model.vocab:
        continue
      if word not in vocab:
        vocab[word] = len(vocab_index)
        number_rep.append(len(vocab_index))
        vocab_index.append(word)
      else:
        number_rep.append(vocab[word])
    dev_df.at[index, sentence] = number_rep   
    
#embedding all vocabulary in a matrix
dimension = 300        
embeddings = 1 * np.random.randn(len(vocab) + 1,dimension)
embeddings[0] = 0 
for word, index in vocab.items():
  if word in word_embed_model.vocab:
    embeddings[index] = word_embed_model[word]


0
100000
0
100000
200000
300000
400000
500000
0
100000


In [0]:
# save all middle files to save time when the colab reinitialized
np.save('/content/gdrive/My Drive/esim_output/embeddings.npy',embeddings)
train_df.to_csv('/content/gdrive/My Drive/esim_output/num_train.csv')
test_df.to_csv('/content/gdrive/My Drive/esim_output/num_test3.csv')
dev_df.to_csv('/content/gdrive/My Drive/esim_output/num_dev.csv')

In [0]:
# find the maximum length in the test and training set
def max_sequence(df1,df2):
    max_seq = max(df1.claim.map(lambda x: len(x)).max(), df1.evidence.map(lambda x: len(x)).max(),df2.claim.map(lambda x: len(x)).max(), df2.evidence.map(lambda x: len(x)).max())
    return max_seq

In [0]:
# define the ESIM model in keras
n_epoch = 4
max_seq = max_sequence(train_df, test_df)
dimension = 300
c_input = Input(shape=(max_seq,), dtype='float32')
e_input = Input(shape=(max_seq,), dtype='float32')
embedding_layer = Embedding(len(embeddings), dimension, weights=[embeddings], input_length=max_seq, trainable=False)
c_embed = embedding_layer(c_input)
e_embed = embedding_layer(e_input)

x1 = Bidirectional(LSTM(100, return_sequences=True))(c_embed)
x2 = Bidirectional(LSTM(100, return_sequences=True))(e_embed)
e = Dot(axes=2)([x1, x2])

e1 = Softmax(axis=2)(e)
e2 = Softmax(axis=1)(e)
e1 = Lambda(K.expand_dims, arguments={'axis' : 3})(e1)
e2 = Lambda(K.expand_dims, arguments={'axis' : 3})(e2)

_x1 = Lambda(K.expand_dims, arguments={'axis' : 1})(x2)
_x1 = Multiply()([e1, _x1])
_x1 = Lambda(K.sum, arguments={'axis' : 2})(_x1)
_x2 = Lambda(K.expand_dims, arguments={'axis' : 2})(x1)
_x2 = Multiply()([e2, _x2])
_x2 = Lambda(K.sum, arguments={'axis' : 1})(_x2)

m1 = Concatenate()([x1, _x1, Subtract()([x1, _x1]), Multiply()([x1, _x1])])
m2 = Concatenate()([x2, _x2, Subtract()([x2, _x2]), Multiply()([x2, _x2])])

y1 = Bidirectional(LSTM(100, return_sequences=True))(m1)
y2 = Bidirectional(LSTM(100, return_sequences=True))(m2)

mx1 = Lambda(K.max, arguments={'axis' : 1})(y1)
av1 = Lambda(K.mean, arguments={'axis' : 1})(y1)
mx2 = Lambda(K.max, arguments={'axis' : 1})(y2)
av2 = Lambda(K.mean, arguments={'axis' : 1})(y2)

y = Concatenate()([av1, mx1, av2, mx2])
y = Dense(128, activation='tanh')(y)
y = Dropout(0.5)(y)
y = Dense(128, activation='tanh')(y)
y = Dropout(0.5)(y)
y = Dense(3, activation='softmax')(y)
model = Model(inputs=[c_input, e_input], outputs=y)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# make sure all training input for the model has the max_seq length
cols = ['claim','evidence']
x = train_df[cols]
y = train_df['label']


v_size = 15000
t_size = len(train_df) - v_size
    
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=v_size)
x_train = {'claim': x_train.claim, 'evidence': x_train.evidence}
x_val = {'claim': x_val.claim, 'evidence': x_val.evidence}

y_train = y_train.values
y_val = y_val.values

for dataset, side in itertools.product([x_train, x_val], ['claim', 'evidence']):
  dataset[side] = pad_sequences(dataset[side], maxlen=max_seq)

y_train = keras.utils.to_categorical(y_train, num_classes=3)
y_val = keras.utils.to_categorical(y_val, num_classes=3)

In [0]:
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type, savepath):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        # val_acc
        plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
        # val_loss
        plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")    
        plt.savefig(savepath)

In [0]:
# train the model and save the checkpoint with best improvement, then make a plot for the training
from keras.callbacks import ModelCheckpoint
history = LossHistory()
model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/esim_output/esim.hdf5', monitor='loss', verbose=1, save_best_only=True)
start = time()
train = model.fit([x_train["claim"], x_train["evidence"]], y_train, epochs=n_epoch, batch_size = 64,validation_data=([x_val["claim"], x_val["evidence"]], y_val),callbacks=[model_checkpoint, history])
print(time()-start)
history.loss_plot("epoch",'/content/gdrive/My Drive/esim_output/esim_plot.jpg')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 135000 samples, validate on 15000 samples
Epoch 1/4
135000/135000 [==============================] - 4965s 37ms/step - loss: 0.8533 - acc: 0.5854 - val_loss: 0.7610 - val_acc: 0.6382

Epoch 00001: loss improved from inf to 0.85330, saving model to /content/gdrive/My Drive/esim_output/esim.hdf5
Epoch 2/4
135000/135000 [==============================] - 4801s 36ms/step - loss: 0.7057 - acc: 0.6723 - val_loss: 0.6782 - val_acc: 0.6928

Epoch 00002: loss improved from 0.85330 to 0.70571, saving model to /content/gdrive/My Drive/esim_output/esim.hdf5
Epoch 3/4
135000/135000 [==============================] - 4921s 36ms/step - loss: 0.5938 - acc: 0.7367 - val_loss: 0.6617 - val_acc: 0.7087

Epoch 00003: loss improved from 0.70571 to 0.59376, saving model to /content/gdrive/My Drive/esim_output/esim.hdf5
Epoch 4/4
135000/135000 [==============================]

In [0]:
# make sure all the test input has the max_seq length
x_test = test_df[cols]
x_test = x_test[:220000]
x_test = {'claim': x_test.claim, 'evidence': x_test.evidence}
for dataset, side in itertools.product([x_test], ['claim', 'evidence']):
  dataset[side] = pad_sequences(dataset[side], maxlen=max_seq)

In [0]:
# predict the reuslt 
predict = model.predict([x_test["claim"], x_test["evidence"]],batch_size=64,verbose =1)
print(predict.shape)

In [0]:
# find the maximum value position in the array
def find_max_pos(array):
  m = max(array)
  p = 0
  for i in range(0,len(array)):
    if (array[i] == m):
      p = i
  return p

In [0]:
# create a new column in the unlabelled test data
test_label = []
for array in predict:
  pos = find_max_pos(array)
  test_label.append(pos)

print(len(test_label))

result_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test-unlabelled.csv')
result_df['label'] = test_label
print(result_df.shape)

In [0]:
# intialized the dictionary to store the output
r = {}
cur_key = ""
for index, row in result_df.iterrows():
  # just update evidence when the key does not change
  if row['key'] == cur_key:
    tmp = []
    tmp.append(row['title'])
    tmp.append(row['page'])
    tmp.append(row['label'])
    r[cur_key]['evidence'].append(tmp)
  
  # when key changed, we need add new member
  else:
      r[cur_key] = {}
      r[cur_key]['claim'] = r[cur_key].setdefault('claim', row['claim'])
      r[cur_key]['evidence'] =  r[cur_key].setdefault('evidence', [])
      tmp = []
      tmp.append(row['title'])
      tmp.append(row['page'])
      tmp.append(row['label'])
      r[cur_key]['evidence'].append(tmp)

# change the format of the dictionary
keys = r.keys()
for key in keys:
  # remove not enough info
  if len(r[key]['evidence']) > 0:
    for item in r[key]['evidence'].copy():
      # if tagged not enough then remove
      if item[2] == 0:
        r[key]['evidence'].remove(item)
    
    if len(r[key]['evidence']) == 0:
      r[key]['label'] = r[key].setdefault('label', "NOT ENOUGH INFO")
      
    else:
      temp = set()
      for item in r[key]['evidence']:
        temp.add(item[2])
      if len(temp) == 2:
        r[key]['label'] = r[key].setdefault('label', "NOT ENOUGH INFO")
        r[key]['evidence'] = []
      elif len(temp) == 1:
        l = list(temp)[0]
        if l == 1:
          r[key]['label'] = "REFUTES"
          for i in range(len(r[key]['evidence'])):
            r[key]['evidence'][i] = r[key]['evidence'][i][:2]
        elif l == 2:
          r[key]['label'] =  "SUPPORTS"
          for i in range(len(r[key]['evidence'])):
            r[key]['evidence'][i] = r[key]['evidence'][i][:2]

In [0]:
import json
json_str = json.dumps(r, indent=4)
with open('/content/gdrive/My Drive/esim_output/testoutput.json', 'w') as json_file:
    json_file.write(json_str)